In [40]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB3, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the CSV files
train_data_df = pd.read_csv('Released_Data/archive_augmented/train_data_new.csv')
super_classes_df = pd.read_csv('Released_Data/superclass_mapping.csv')
sub_classes_df = pd.read_csv('Released_Data/subclass_mapping.csv')

# Rename
super_classes_df.rename(columns={'class': 'superclass_name'}, inplace=True)
sub_classes_df.rename(columns={'class': 'subclass_name'}, inplace=True)

# Merge the class names with the training data
train_data_df = train_data_df.merge(super_classes_df, left_on='superclass_index', right_on='index', how='left')
train_data_df = train_data_df.merge(sub_classes_df, left_on='subclass_index', right_on='index', how='left')

# Superclass_name (can replace with subclass_name depending on classification task)
train_data_df['class'] = train_data_df['superclass_name']

# Split the dataset into training and validation sets
train_df, validation_df = train_test_split(train_data_df, test_size=0.2)

# Initialize the ImageDataGenerator with EfficientNet's preprocess_input
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Image folder path
image_folder_path = 'Released_Data/train_shuffle_augmented'

# Load and preprocess images for training
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=image_folder_path,
    x_col='image',
    y_col='class',
    class_mode='categorical',
    target_size=(300, 300),
    batch_size=32
)

# Load and preprocess images for validation
validation_generator = datagen.flow_from_dataframe(
    dataframe=validation_df,
    directory=image_folder_path,
    x_col='image',
    y_col='class',
    class_mode='categorical',
    target_size=(300, 300),
    batch_size=32
)

# Load pre-trained EfficientNetB3
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Freeze the base model
base_model.trainable = False

# Add custom top layers
x = GlobalAveragePooling2D()(base_model.output)
output = Dense(len(super_classes_df), activation='softmax')(x) # Number of classes
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Found 5093 validated image filenames belonging to 4 classes.
Found 1274 validated image filenames belonging to 4 classes.


In [41]:
# import os

# # Folder containing the jpg files
# folder_path = 'Released_Data/novel_data_augmented'
# files = sorted(os.listdir(folder_path))

# # Starting number for renaming
# start_number = 6322

# # Check if the number of files is correct
# if len(files) != 45:
#     print("Error: There should be exactly 45 files in the folder.")
# else:
#     for i, file in enumerate(files):
#         # Construct the new file name
#         new_file_name = f"{start_number + i}.jpg"

#         # Construct full file paths
#         old_file_path = os.path.join(folder_path, file)
#         new_file_path = os.path.join(folder_path, new_file_name)

#         # Rename the file
#         os.rename(old_file_path, new_file_path)

#     print("Files have been renamed successfully.")


In [42]:
# Train the model
model.fit(train_generator, validation_data=validation_generator, epochs=10, batch_size=32)

# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Re-compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training
model.fit(train_generator, validation_data=validation_generator, epochs=10, batch_size=32)


Epoch 1/10
160/160 [==============================] - 1720s 10s/step - loss: 0.5716 - accuracy: 0.7907 - val_loss: 0.3622 - val_accuracy: 0.8705
Epoch 2/10
160/160 [==============================] - 1647s 10s/step - loss: 0.3547 - accuracy: 0.8771 - val_loss: 0.3061 - val_accuracy: 0.8893
Epoch 3/10
160/160 [==============================] - 1642s 10s/step - loss: 0.3067 - accuracy: 0.8881 - val_loss: 0.2624 - val_accuracy: 0.9129
Epoch 4/10
160/160 [==============================] - 1601s 10s/step - loss: 0.2798 - accuracy: 0.9071 - val_loss: 0.2759 - val_accuracy: 0.8925
Epoch 5/10
160/160 [==============================] - 1642s 10s/step - loss: 0.2556 - accuracy: 0.9132 - val_loss: 0.2411 - val_accuracy: 0.9215
Epoch 6/10
160/160 [==============================] - 1567s 10s/step - loss: 0.2492 - accuracy: 0.9132 - val_loss: 0.2380 - val_accuracy: 0.9152
Epoch 7/10
160/160 [==============================] - 1005s 6s/step - loss: 0.2368 - accuracy: 0.9179 - val_loss: 0.2184 - val_acc

In [43]:
test_image_folder = 'Released_Data/test_shuffle'   # Correct path to your test images
image_files = [os.path.join(test_image_folder, img) for img in os.listdir(test_image_folder) if img.endswith('.jpg')]
test_df = pd.DataFrame(image_files, columns=['filename'])


In [44]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [45]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col=None,
    target_size=(300, 300),
    batch_size=32,
    class_mode=None,
    shuffle=False
)


Found 12377 validated image filenames.


In [46]:
predictions = model.predict(test_generator)


387/387 [==============================] - 1818s 5s/step


In [47]:
predicted_superclass_indices = np.argmax(predictions, axis=1)


In [48]:
image_filenames = [f"{i}.jpg" for i in range(len(predictions))]
results_df = pd.DataFrame({
    'ID': image_filenames,
    'Target': predicted_superclass_indices
})
results_df.head()

,ID,Target
0,0.jpg,1
1,1.jpg,1
2,2.jpg,1
3,3.jpg,0
4,4.jpg,0


In [49]:
results_df.to_csv("res/superclass_pred_augmented.csv", index=False)